In [1]:
import gymnasium as gym
from pogema import GridConfig
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext autoreload
%autoreload 2
%matplotlib inline

grid_config = GridConfig(
    size=16,
    density=0.5,
    num_agents=1,
    max_episode_steps=64
)

env = gym.make("Pogema-v0",grid_config=grid_config)

a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


#### Optuna Integration

In [2]:
""" Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using A2C implementation from Stable-Baselines3
on a Gymnasium environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python sb3_simple.py

"""
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.0e4)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "verbose": 1,
    "env": env
}

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for A2C hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)

    trial.set_user_attr("gamma", gamma)

    return {
        "gamma": gamma,
        "learning_rate": learning_rate,
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model.
    model = A2C(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, n_jobs=4, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))


/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-10 17:19:25,386] A new study created in memory with name: no-name-52c464f8-98df-4fb4-8bb4-2184d70c7a25
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 38       |
|    ep_rew_mean        | 0.769    |
| time/                 |          |
|    fps                | 207      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.61    |
|    explained_variance | 0.993    |
|    learning_rate      | 1.01e-05 |
|    n_updates          | 99       |
|    policy_loss        | -0.436   |
|    value_loss         | 0.0738   |
--------------------------------

[I 2023-11-10 17:20:20,319] Trial 1 finished with value: 0.3333333333333333 and parameters: {'gamma': 0.0006524574108032467, 'lr': 2.3087090920528374e-05}. Best is trial 1 with value: 0.3333333333333333.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| eval/                 |           |
|    mean_ep_length     | 81.7      |
|    mean_reward        | 0         |
| time/                 |           |
|    total_timesteps    | 10000     |
| train/                |           |
|    entropy_loss       | -4.02e-08 |
|    explained_variance | 5.96e-08  |
|    learning_rate      | 0.474     |
|    n_updates          | 1999      |
|    policy_loss        | -0        |
|    value_loss         | 4.75e-06  |
-------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 76.7     |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 181      |
|    iterations      | 2000     |
|    time_elapsed    | 55       |
|    total_timesteps | 10000    |
---------------------------------


[I 2023-11-10 17:20:20,500] Trial 0 finished with value: 0.0 and parameters: {'gamma': 0.0004218386393968535, 'lr': 0.47353318794973187}. Best is trial 1 with value: 0.3333333333333333.


...

------------------------------------
| eval/                 |          |
|    mean_ep_length     | 36.7     |
|    mean_reward        | 0.333    |
| time/                 |          |
|    total_timesteps    | 5000     |
| train/                |          |
|    entropy_loss       | -0.00129 |
|    explained_variance | 0.0777   |
|    learning_rate      | 0.00449  |
|    n_updates          | 999      |
|    policy_loss        | 1.05e-06 |
|    value_loss         | 9.97e-05 |
------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50.5     |
|    ep_rew_mean     | 0.429    |
| time/              |          |
|    fps             | 238      |
|    iterations      | 1000     |
|    time_elapsed    | 20       |
|    total_timesteps | 5000     |
---------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 48.4      |
|    ep_rew_mean        | 0.

[I 2023-11-10 17:29:43,994] Trial 59 pruned. 


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 55.3      |
|    ep_rew_mean        | 0.34      |
| time/                 |           |
|    fps                | 269       |
|    iterations         | 1200      |
|    time_elapsed       | 22        |
|    total_timesteps    | 6000      |
| train/                |           |
|    entropy_loss       | -0.00118  |
|    explained_variance | 0         |
|    learning_rate      | 0.00449   |
|    n_updates          | 1199      |
|    policy_loss        | -1.49e-06 |
|    value_loss         | 0.000235  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 53.7     |
|    ep_rew_mean        | 0.32     |
| time/                 |          |
|    fps                | 288      |
|    iterations         | 1300     |
|    time_elapsed       | 22       |
|    total_timesteps    | 6500     |
| train/             

[I 2023-11-10 17:29:46,456] Trial 58 finished with value: 0.0 and parameters: {'gamma': 0.07780183194607228, 'lr': 0.004490940691248326}. Best is trial 5 with value: 1.0.


Number of finished trials:  60
Best trial:
  Value:  1.0
  Params: 
    gamma: 0.010626448733732002
    lr: 0.0013429429973806648
  User attrs:
    gamma: 0.989373551266268


### Train agent with best hyper parameters

In [8]:
a2c_model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    gamma=1-0.005090895994889371,
    learning_rate=0.03807432884729837,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
# Train agent and save it
a2c_model.learn(int(1.0e4))
a2c_model.save("saved/a2c_b_mini")

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 64        |
|    ep_rew_mean        | 0         |
| time/                 |           |
|    fps                | 1721      |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -5.05e-06 |
|    explained_variance | 0         |
|    learning_rate      | 0.0381    |
|    n_updates          | 99        |
|    policy_loss        | -0        |
|    value_loss         | 2.01e-08  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 60.1      |
|    ep_rew_mean        | 0.0625    |
| time/                 |           |
|    fps                | 1764      |
|    iterations         | 200       |
|    time_elapsed       | 0         |
|    total_timesteps    | 1000      |
| train/    

#### Testing 

In [10]:
a2c_model = A2C.load("saved/a2c_b_mini")

env.reset()

mean_reward, std_reward = evaluate_policy(a2c_model, env, deterministic=True, n_eval_episodes=20)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/aditya/miniconda3/envs/pogema/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:0.20 +/- 0.40


In [11]:
# RANDOM SEED
from IPython.display import SVG, display
from pogema.animation import AnimationMonitor, AnimationConfig

env = AnimationMonitor(env)

def evaluate_success_rate(model, env, num_episodes=1000):
    success_count = 0
    step_array = []
    for i in range(num_episodes):
        print(f'---{i}---')
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 64
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                env.save_animation(f"render{i}.svg", AnimationConfig(egocentric_idx=0))
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(a2c_model, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

---0---
4 64 0 False
4 63 0 False
4 62 0 False
4 61 0 False
4 60 0 False
4 59 0 False
4 58 0 False
4 57 0 False
4 56 0 False
4 55 0 False
4 54 0 False
4 53 0 False
4 52 0 False
4 51 0 False
4 50 0 False
4 49 0 False
4 48 0 False
4 47 0 False
4 46 0 False
4 45 0 False
4 44 0 False
4 43 0 False
4 42 0 False
4 41 0 False
4 40 0 False
4 39 0 False
4 38 0 False
4 37 0 False
4 36 0 False
4 35 0 False
4 34 0 False
4 33 0 False
4 32 0 False
4 31 0 False
4 30 0 False
4 29 0 False
4 28 0 False
4 27 0 False
4 26 0 False
4 25 0 False
4 24 0 False
4 23 0 False
4 22 0 False
4 21 0 False
4 20 0 False
4 19 0 False
4 18 0 False
4 17 0 False
4 16 0 False
4 15 0 False
4 14 0 False
4 13 0 False
4 12 0 False
4 11 0 False
4 10 0 False
4 9 0 False
4 8 0 False
4 7 0 False
4 6 0 False
4 5 0 False
4 4 0 False
4 3 0 False
4 2 0 False
4 1 0 False
---1---
4 64 0 False
4 63 0 False
4 62 0 False
4 61 0 False
4 60 0 False
4 59 0 False
4 58 0 False
4 57 0 False
4 56 0 False
4 55 0 False
4 54 0 False
4 53 0 False
4 52 